In [22]:
#importing libraries
import numpy as np
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

In [23]:
#importing the shapefile
gply = gpd.read_file('Stadtteile/Düsseldorf_Stadtteile.shp')

In [24]:
#dropping irrelevant columne
columns_drop = ["OBJECTID", "Quelle", "Stand", "Stadtbezir", "Stadtteil", "SHAPE_Leng", "SHAPE_Area"]
gply.drop(labels = columns_drop, axis = "columns", inplace = True)

In [4]:
gply.dtypes

Name          object
geometry    geometry
dtype: object

In [25]:
#converting to standard crs
gply = gply.to_crs(4326)
gply.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
gply.head()

,Name,geometry
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ..."
1,Angermund,"POLYGON Z ((6.80952 51.34292 0.00000, 6.80943 ..."
2,Lörick,"POLYGON Z ((6.74610 51.25203 0.00000, 6.74380 ..."
3,Heerdt,"POLYGON Z ((6.73560 51.23767 0.00000, 6.73399 ..."
4,Oberkassel,"POLYGON Z ((6.76868 51.23195 0.00000, 6.76868 ..."


In [27]:
#loading the noise dataset 
df = pd.read_csv('noise_c.csv')
df.rename(columns={'Bezirk': 'Address'}, inplace=True)

In [28]:
df.dtypes

Address       object
DB_TAG         int64
DB_NACHT       int64
Latitude     float64
Longitude    float64
dtype: object

In [29]:
#converting the noise dataset to GeoDataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude),crs='4326')

In [30]:
gdf.shape,gdf.crs

((77018, 6),
 <Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich)

In [31]:
gdf.head()

,Address,DB_TAG,DB_NACHT,Latitude,Longitude,geometry
0,"Aachener Straße 1, Düsseldorf, Germany",68,62,51.207592,6.776500,POINT (6.77650 51.20759)
1,"Aachener Straße 10, Düsseldorf, Germany",68,61,51.207128,6.776436,POINT (6.77644 51.20713)
2,"Aachener Straße 101, Düsseldorf, Germany",70,63,51.202533,6.771146,POINT (6.77115 51.20253)
3,"Aachener Straße 103, Düsseldorf, Germany",70,62,51.202428,6.771077,POINT (6.77108 51.20243)
4,"Aachener Straße 105, Düsseldorf, Germany",69,62,51.202332,6.771107,POINT (6.77111 51.20233)


In [32]:
#merging both datasets to plot choropleth
merge=gpd.sjoin(gply,gdf,how='inner',op='contains')
merge.head()

C:\Users\joe16\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,Name,geometry,index_right,Address,DB_TAG,DB_NACHT,Latitude,Longitude
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...",65394,"Schulweg 14, Düsseldorf, Germany",50,41,51.324019,6.738590
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...",40624,"Kalkstraße 16, Düsseldorf, Germany",50,42,51.324031,6.742087
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...",13616,"Bockumer Straße 49c, Düsseldorf, Germany",61,52,51.324051,6.739528
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...",8210,"An der Schanz 7, Düsseldorf, Germany",51,42,51.324113,6.741296
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...",8202,"An der Schanz 10, Düsseldorf, Germany",54,45,51.324137,6.740668


In [33]:
gba=merge.groupby(['Name']).agg({'DB_TAG':'mean','DB_NACHT':'mean'})
gba.shape,gba.head()

((50, 2),
               DB_TAG   DB_NACHT
 Name                           
 Altstadt   57.699789  48.217759
 Angermund  51.446281  42.895741
 Benrath    58.261646  50.305124
 Bilk       59.179613  51.136587
 Carlstadt  60.528736  51.890805)

In [34]:
gply.head()

,Name,geometry
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ..."
1,Angermund,"POLYGON Z ((6.80952 51.34292 0.00000, 6.80943 ..."
2,Lörick,"POLYGON Z ((6.74610 51.25203 0.00000, 6.74380 ..."
3,Heerdt,"POLYGON Z ((6.73560 51.23767 0.00000, 6.73399 ..."
4,Oberkassel,"POLYGON Z ((6.76868 51.23195 0.00000, 6.76868 ..."


In [35]:
gply.set_index('Name',inplace=True)

In [36]:
gply['Avg_Day']=gba['DB_TAG']
gply['Avg_Night']=gba['DB_NACHT']

In [37]:
gply.shape,gply.head()

((50, 3),
                                                      geometry    Avg_Day  \
 Name                                                                       
 Wittlaer    POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...  53.339965   
 Angermund   POLYGON Z ((6.80952 51.34292 0.00000, 6.80943 ...  51.446281   
 Lörick      POLYGON Z ((6.74610 51.25203 0.00000, 6.74380 ...  53.602421   
 Heerdt      POLYGON Z ((6.73560 51.23767 0.00000, 6.73399 ...  60.602572   
 Oberkassel  POLYGON Z ((6.76868 51.23195 0.00000, 6.76868 ...  58.515371   
 
             Avg_Night  
 Name                   
 Wittlaer    45.046811  
 Angermund   42.895741  
 Lörick      45.645251  
 Heerdt      53.483601  
 Oberkassel  50.722032  )

In [38]:
gply.reset_index(inplace=True)

In [39]:
adf=gpd.GeoDataFrame(gply,crs='4326')
adf.head()

,Name,geometry,Avg_Day,Avg_Night
0,Wittlaer,"POLYGON Z ((6.76623 51.31355 0.00000, 6.76617 ...",53.339965,45.046811
1,Angermund,"POLYGON Z ((6.80952 51.34292 0.00000, 6.80943 ...",51.446281,42.895741
2,Lörick,"POLYGON Z ((6.74610 51.25203 0.00000, 6.74380 ...",53.602421,45.645251
3,Heerdt,"POLYGON Z ((6.73560 51.23767 0.00000, 6.73399 ...",60.602572,53.483601
4,Oberkassel,"POLYGON Z ((6.76868 51.23195 0.00000, 6.76868 ...",58.515371,50.722032


In [40]:
map_1 = KeplerGl(height=500)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [41]:
map_1.add_data(data=adf, name='kep_ch')
map_1

KeplerGl(data={'kep_ch': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …

In [22]:
map_1.save_to_html(file_name='kep_choro.html')

Map saved to kep_choro.html!
